#### Author: Faolán Hamilton

## Part 1 70%
Write a jupyter notebook that analyses the differences between the sexes by age in Ireland.

- Weighted mean age (by sex)
- The difference between the sexes by age
This part does not need to look at the regions.

i.e. You can take the notebook I used in the lectures and substitute the sexes for the regions.

## Part 2 20%
- In the same notebook, make a variable that stores an age (say 35).

- Write that code that would group the people within 5 years of that age together, into one age group 

- Calculate the population difference between the sexes in that age group.

## Part 3 10%
In the same notebook.

- Write the code that would work out which region in Ireland has the biggest population difference between the sexes in that age group

--------------------

## Part 1
- This notebook is for assignment 5 of the Programming for Data Analytics module. This structure has been edited from the 'weighted stats' notebook created by Andrew Beatty


In [ ]:
# The first step is bringing in modules used in this notebook
import pandas as pd

In [ ]:
#This is the dataset with the ages broken down by sex in Ireland
url = "C:\Users\ClaireReilly\Downloads"
#Read in the url as a csv
df = pd.read_csv(url)
#Show the last three rows of data
df.tail(3)

The data currently list out all Local Authorities in the output. For the first section, I want to create a subset of this dataset using only Ireland level data

In [ ]:
#We only need data for Ireland, so the other rows can be removed for now
df = df[df["Administrative Counties"] == "Ireland"]
df.tail(3)

There are a lot of columns in the dataset, but we only need 'Sex', Single Year of Age' and 'VALUE'

In [ ]:
# Show all the columns in the dataset to identify which to remove
headers = df.columns.tolist()
headers

In [ ]:
#remove the headers that are not relevant
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)
df

In [ ]:
#improve the naming convention of the 'Single Year of Age'
df = df[df["Single Year of Age"] != "All ages"]
#Change 'Under 1 year' to '0'
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\\D', '', regex=True)
df

In [ ]:
#Ensure 'Single Year of Age' and 'VALUE' are integers
df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')
#print (df.head(3))
df.info()
df

I wanted to ensure the csv output values were rounded to whole values and found this Stackoverflow (https://stackoverflow.com/questions/76584654/python-pandas-to-csv-is-rounding-any-way-for-it-to-not-round)

In [ ]:
df_analysis = pd.pivot_table(df,'VALUE',"Single Year of Age", 'Sex')
# write out the entire file to local machine
df_analysis.to_csv(f"assignment05_population_for_analysis.csv", float_format='%.0f')
print (df_analysis.head(3))

---------------------------------------------------------------

## Now we can do weighted descriptive statistics

In [ ]:
headers = list(df_analysis.columns)
district = headers[0]
district

Weighted mean is sum(age*population at age) / sum (populations at age)

In [ ]:
number_people = df_analysis[district].sum()
number_people


In [ ]:
df_analysis

In [ ]:
cumages = df_analysis[district].mul(df_analysis.index, axis=0).sum()
cumages

In [ ]:
weighted_mean = cumages/number_people
weighted_mean

#### Or you can use numpy

In [ ]:
import numpy as np
w_mean = np.average(df_analysis.index, weights=df_analysis[district])
w_mean

### weighted median

    create a series of the cumlative sums and find the index of the middle value

In [ ]:
cumsum = df_analysis[district].cumsum()
cumsum

In [ ]:
# sum of all the number of people
cutoff = df_analysis[district].sum()/2
cutoff

In [ ]:
df_analysis[district][cumsum>=cutoff].index[0]

## Weighted standard deviation

The is the same formula as the normal standard deviation, just with weights applied to the differenced to the weight mean

In [ ]:
# incorrect way
df_analysis[district].std()

In [ ]:
# incorrect
df_analysis[district].mean()

We need the weighted std

In [ ]:
import numpy as np
w_mean = np.average(df_analysis.index, weights=df_analysis[district])
w_mean

In [ ]:
w_variance = np.average((df_analysis.index - w_mean)**2, weights= df_analysis[district])
w_variance

In [ ]:
w_std = np.sqrt(w_variance)
w_std